# SQL 고급 활용 및 튜닝

# 제 1장 아키텍처 기반 튜닝 원리

## 제1절 데이터베이스 아키텍처

### 1. 아키텍처 개관

### 가. ORACLE 아키텍처

- 데이터베이스 : 물리적인 디스크에 저장된 데이터집합 (데이터파일, 리두로그파일, 컨트롤파일)
- 인스턴스 : 공유메모리(SGA)와 이를 엑세스하는 프로세스 집합

#### 하나의 인스턴스는 하나의 데이터베이스를 액세스(Single)
#### 여러개의 인스턴스는 하나의 데이터베이스를 엑세스(RAC)

### 나. SQL Server 아키텍처

In [ ]:
- 하나의 인스턴스당 최고 32,767개의 데이터베이스를 정의해서 사용
- 

## 제2절 SQL 파싱 부하

## 제3절 데이터베이스 Call과 네트워크 부하

## 제4절 데이터베이스 I/O 원리